code from https://github.com/TiKeil/Trust-region-TSRBLOD-code/blob/main/scripts/FINAL_exp_1.py

In [1]:
import numpy as np
from matplotlib import pyplot as plt

from pymor.core.logger import set_log_levels
from pymor.core.defaults import set_defaults
from pymor.core.cache import disable_caching
from pdeopt.tools import print_iterations_and_walltime
set_log_levels({'pymor': 'ERROR',
                'notebook': 'INFO'})
%matplotlib inline

def prepare_kernels():
    set_log_levels({'pymor': 'WARN'})
    # important for the estimator
    set_defaults({"pymor.algorithms.gram_schmidt.gram_schmidt.rtol": 1e-4})
    set_defaults({"pymor.algorithms.gram_schmidt.gram_schmidt.check": False})
    disable_caching()

use_pool = False
if use_pool:
    from pymor.parallel.mpi import MPIPool
    pool = MPIPool()
    store_in_tmp = 'tmp'  # <---- adjust this depending on your HPC system 
else:
    from pymor.parallel.dummy import DummyPool
    pool = DummyPool()
    store_in_tmp = False
pool.apply(prepare_kernels)
print_on_ranks = True

In [2]:
coarse_elements = 20
n = 200 # 1200
diameter = np.sqrt(2)/n

two_scale_estimator_for_RBLOD = False
save_correctors = False

use_FEM = True
#use_FEM = False
use_fine_mesh = True
#use_fine_mesh = False

# skip_estimator = False
skip_estimator = True

add_error_residual = True
# add_error_residual = False

from pdeopt.problems import large_thermal_block
from pdeopt.discretizer import discretize_quadratic_NCD_pdeopt_stationary_cg
from pdeopt.discretize_gridlod import discretize_gridlod
from pdeopt.discretize_gridlod import discretize_quadratic_pdeopt_with_gridlod

high_conductivity, low_conductivity, min_diffusivity, rhs_value = 4., 1.2, 1., 10.
first_factor, second_factor = 4, 8

print(f'\nVARIABLES: \n'
      f'Coarse elements:        {coarse_elements} x {coarse_elements}\n'
      f'Fine elements:          {n} x {n}\n'
      f'high_c/low_c/min_c:     {high_conductivity}/{low_conductivity}/{min_diffusivity}\n'
      f'rhs/f_1/f_2:            {rhs_value}/{first_factor}/{second_factor}\n')

global_problem, world, local_problem_constructer, f, aFines, f_fine = \
    large_thermal_block(diameter, coarse_elements, blocks=(4, 4), plot=False,
                        return_fine=use_FEM, high_conductivity=high_conductivity,
                        low_conductivity=low_conductivity, rhs_value=rhs_value,
                        first_factor=first_factor, second_factor=second_factor,
                        min_diffusivity=min_diffusivity)

domain_of_interest = None

problem = global_problem

mu_d = global_problem.parameter_space.sample_randomly(1, seed=23)[0]
mu_d_array = mu_d.to_numpy()

# making it harder for the optimizer
for i in [3,4,6,7,8,9,11,14]:
    try:
        mu_d_array[i] = high_conductivity
    except:
        pass
for i in [3,4,5,6]:
    try:
        mu_d_array[i+25] = low_conductivity
    except:
        pass

mu_d = mu_d.parameters.parse(mu_d_array)
norm_mu_d = np.linalg.norm(mu_d_array)
# mu_d = None
mu_for_tikhonov = mu_d

sigma_u = 100
weights = {'sigma_u': sigma_u, 'diffusion': 0.001,
           'low_diffusion': 0.001}

# optional_enrichment = True

/home/tim/dune-gdt-ipl-in-pymor/pymor/src/pymor/core/config.py:46: UserWarning: dune-gdt bindings have been tested for version 2022.1.x (x >= 1) (installed: 0+untagged.5325.g119d764).
  warnings.warn('dune-gdt bindings have been tested for version 2022.1.x (x >= 1) '
/home/tim/dune-gdt-ipl-in-pymor/pymor/src/pymor/core/config.py:54: UserWarning: dune-gdt bindings have been tested for dune-xt 2022.1.x (x >= 4) (installed: 2021.1.9+320.gd3920295.dirty).
  warnings.warn('dune-gdt bindings have been tested for dune-xt 2022.1.x (x >= 4) '



VARIABLES: 
Coarse elements:        20 x 20
Fine elements:          200 x 200
high_c/low_c/min_c:     4.0/1.2/1.0
rhs/f_1/f_2:            10.0/4/8



In [3]:
u_d = None
mu_for_u_d = mu_d
desired_temperature = None

In [4]:
# u_d = None
# mu_for_u_d = None
# desired_temperature = 0.1

In [5]:
coarse_J = False
if coarse_J is False:
    assert use_fine_mesh
    N_coarse = None
else:
    N_coarse = coarse_elements

In [6]:
opt_fom, data, mu_bar = \
    discretize_quadratic_NCD_pdeopt_stationary_cg(problem,
                                                  diameter,
                                                  weights.copy(),
                                                  domain_of_interest=domain_of_interest,
                                                  desired_temperature=desired_temperature,
                                                  mu_for_u_d=mu_for_u_d,
                                                  mu_for_tikhonov=mu_for_tikhonov,
                                                  coarse_functional_grid_size=N_coarse,
                                                  u_d=u_d)

### counting evaluations in opt_fom
from pdeopt.tools import EvaluationCounter, LODEvaluationCounter
counter = EvaluationCounter()

opt_fom = opt_fom.with_(evaluation_counter=counter)

constant  0.002151404163076754


In [7]:
'''
    Variables for the optimization algorithms
'''

seed = 1                   # random seed for the starting value
radius = 0.1               # TR radius 
FOC_tolerance = 1e-4       # tau_FOC
sub_tolerance = 1e-8       # tau_sub
safety_tol = 1e-14         # Safeguard, to avoid running the optimizer in machine prevision
max_it = 60                # Maximum number of iteration for the TR algorithm
max_it_sub = 100           # Maximum number of iteration for the TR optimization subproblem
max_it_arm = 50            # Maximum number of iteration for the Armijo rule
init_step_armijo = 0.5     # Initial step for the Armijo rule
armijo_alpha = 1e-4        # kappa_arm
beta = 0.95                # beta_2
epsilon_i = 1e-8           # Treshold for the epsilon active set (Kelley '99)
control_mu = False

# some implementational variables
Qian_Grepl_subproblem = True

reductor_type = 'simple_coercive'
LOD_reductor_type = 'coercive'
if skip_estimator:
    relaxed_reductor_type = 'non_assembled'
    relaxed_LOD_reductor_type = 'non_assembled'
    relaxed_add_error_residual = False
else:
    relaxed_reductor_type = 'simple_coercive'
    relaxed_LOD_reductor_type = 'coercive'
    relaxed_add_error_residual = True

# starting with 
parameter_space = global_problem.parameter_space
mu = parameter_space.sample_randomly(1, seed=seed)[0]

# ### What methods do you want to test ?

optimization_methods = [
    # FOM Method
      'BFGS',
#       'BFGS_LOD',
      'BFGS_IPL',
    # TR-RB
        # NCD-corrected from KMSOV'20
#           'Method_RB', # TR-RB
        # localized BFGS
#           'Method_RBLOD',
#           'Method_TSRBLOD',
            'Method_LRBMS',
    # R TR Methods
      'Method_R_TR'
]

In [8]:
parameters = opt_fom.parameters
if mu_for_u_d is not None:
    mu_opt = mu_d
else:
    # adapt this if needed! 
    mu_opt = parameters.parse([1., 2.05833162, 2.41529224, 3.06932311,
                               3.37045877, 2.14031204, 1.2, 1.2])

print('Starting parameter: ', mu.to_numpy())
# J_start_LOD = gridlod_opt_fom.output_functional_hat(mu, pool=pool)
if use_FEM:
    J_start = opt_fom.output_functional_hat(mu)
    print('Starting J FEM: ', J_start)
else:
    J_start = J_start_LOD
    J_opt = J_opt_LOD
# print('Starting J LOD: ', J_start_LOD)
print()
    
mu_opt_as_array = mu_opt.to_numpy()
# J_opt_LOD = gridlod_opt_fom.output_functional_hat(mu_opt, pool=pool)
print('Optimal parameter: ', mu_opt_as_array)
print('Norm mu_d: ', norm_mu_d)
if use_FEM:
    J_opt = opt_fom.output_functional_hat(mu_opt)
    print('Optimal J FEM: ', J_opt)
# print('Optimal J LOD: ', J_opt_LOD)

print('\nParameter space: ', mu_opt.parameters)

Starting parameter:  [2.25106601 3.16097348 1.00034312 1.90699772 1.44026767 1.27701578
 1.55878063 2.03668218 2.19030242 2.6164502  2.25758354 3.0556585
 1.61335675 3.63435231 1.08216278 3.01140253 2.25191441 2.67606949
 1.42116082 1.59430447 3.40223371 3.90478473 1.94027253 3.07696785
 1.17527783 1.17892133 1.01700884 1.00781096 1.03396608 1.1756285
 1.01966937 1.08422153]
Starting J FEM:  1.0508730423458459

Optimal parameter:  [2.55189365 3.84088781 3.29637928 4.         4.         3.05866626
 4.         4.         4.         4.         1.00739464 4.
 3.65484262 1.90122907 4.         3.93528075 3.53528147 1.19522632
 1.88423339 1.86380332 3.46739902 2.87854911 1.33143314 1.00158643
 1.18843325 1.02830015 1.08431931 1.06929789 1.2        1.2
 1.2        1.2       ]
Norm mu_d:  16.048194664888687
Optimal J FEM:  1.0

Parameter space:  {diffusion: 24, low_diffusion: 8}


In [9]:
from pdeopt.tools import compute_errors
from pdeopt.TR import solve_optimization_subproblem_BFGS

counter.reset_counters()
TR_parameters = {'radius': 1.e18, 'sub_tolerance': FOC_tolerance, 
                 'max_iterations_subproblem': 500,
                 'starting_parameter': mu,
                 'epsilon_i': epsilon_i,
                 'max_iterations_armijo': max_it_arm,
                 'initial_step_armijo': init_step_armijo,
                 'armijo_alpha': armijo_alpha,
                 'full_order_model': True}

if 'BFGS' in optimization_methods or 'All' in optimization_methods:
    print("\n________________ FOM BFGS________________________\n")
    muoptfom,_,_,_, times_FOM, mus_FOM, Js_FOM, FOC_FOM = \
        solve_optimization_subproblem_BFGS(opt_fom, parameter_space, mu,
                                           TR_parameters, timing=True, FOM=True)
    times_full_FOM, J_error_FOM, mu_error_FOM, FOC = \
        compute_errors(opt_fom, parameter_space, J_start, J_opt, mu, mu_opt_as_array,
                       mus_FOM, Js_FOM, times_FOM, 0, FOC_FOM)
    times_full_FOM = times_full_FOM[1:]


________________ FOM BFGS________________________

Starting projected BFGS method
Starting parameter {diffusion: [2.251066014107722, 3.1609734803264744, 1.0003431244520347, 1.9069977178955193, 1.440267672451339, 1.2770157843063934, 1.5587806341330128, 2.036682181129143, 2.19030242269201, 2.616450202010071, 2.2575835432098845, 3.0556585011902784, 1.6133567491945522, 3.6343523091728365, 1.0821627795937785, 3.011402530535207, 2.251914407101381, 2.676069485337255, 1.4211608157857012, 1.5943044672546365, 3.40223370602661, 3.9047847271581926, 1.9402725344777285, 3.0769678470079422], low_diffusion: [1.1752778304592075, 1.1789213327007695, 1.0170088422739556, 1.0078109566465765, 1.0339660839129137, 1.1756285006858826, 1.01966936676661, 1.0842215250010103]}
Step [2.25271811 3.16299183 1.00693427 1.91334066 1.44736604 1.28332595
 1.56310567 2.04054667 2.19440831 2.620182   2.25874738 3.05905965
 1.61709557 3.63431804 1.08689836 3.01411222 2.253871   2.67525615
 1.4231056  1.59604441 3.40328    

Step [2.64280572 3.73060682 3.50770183 4.         4.         3.35443943
 3.93746227 3.99000701 3.97024008 4.         1.29612396 4.
 3.44137099 2.08656718 3.73007201 3.31308485 3.32145235 1.27403747
 1.68830375 1.68948744 3.30044799 2.82801544 1.2694171  1.092843
 1.11487325 1.11208465 1.2        1.2        1.2        1.11621988
 1.2        1.2       ], functional 1.0005811849915753 , FOC condition 0.0012336883164632611
Step [2.65759144 3.75163074 3.57494169 3.99999868 4.         3.4158676
 3.98829266 4.         4.         4.         1.29567591 4.
 3.48384018 2.07290857 3.78676876 3.38323449 3.34488609 1.25632206
 1.70500374 1.70323393 3.30760307 2.81893643 1.26304278 1.06470465
 1.14579483 1.1422936  1.19992696 1.19991208 1.2        1.14737979
 1.2        1.2       ], functional 1.00051780345669 , FOC condition 0.0011291984851329352
Step [2.66487939 3.76695666 3.61690137 4.         3.9999784  3.45149542
 4.         3.99999511 4.         4.         1.28416046 4.
 3.51449994 2.05228563 3

Step [2.55984785 3.87211613 3.34073666 3.97837587 4.         3.10704806
 3.91573688 4.         3.963672   3.9987933  1.01394604 4.
 3.64622087 1.89589371 4.         3.94781754 3.52709317 1.18946713
 1.88150418 1.86130542 3.44947517 2.86167887 1.31001703 1.01907731
 1.06451482 1.11459051 1.0595737  1.16032987 1.2        1.10731941
 1.2        1.18109017], functional 1.0000296574108016 , FOC condition 0.0002537915198721875
Step [2.56019829 3.87031559 3.34025009 3.97464792 4.         3.10717427
 3.90584567 4.         3.94884758 3.9977811  1.01656038 4.
 3.64533417 1.89876108 4.         3.94221368 3.52638865 1.19150676
 1.88210698 1.8620921  3.45032389 2.86360999 1.31124107 1.01802142
 1.08022288 1.10969214 1.04210404 1.14253741 1.2        1.12387421
 1.2        1.17474583], functional 1.0000267896081838 , FOC condition 0.00024490301024675265
Step [2.5596484  3.87130613 3.34032996 3.97989493 4.         3.10578244
 3.89748722 4.         3.9355039  3.99754156 1.01249392 4.
 3.65108289 1.8962

In [10]:
if 'BFGS' in optimization_methods or 'All' in optimization_methods:
    print("\n________________ FOM BFGS________________________\n")
    BFGS_dict = counter.print_result(True)
    # print_RB_result(BFGS_dict)
    print_iterations_and_walltime(len(times_full_FOM), times_full_FOM[-1])
    print('mu_error: ', mu_error_FOM[-1]) 
counter.reset_counters()


________________ FOM BFGS________________________


FEM solves:   157
RB solves:    0


outer iterations: 51
total walltime:   55.10 seconds
mu_error:  0.09136933573861761


In [11]:
'''
    ROM OPTIMIZATION ALGORITHMS
'''

import time
from pdeopt.model import build_initial_basis
from pdeopt.reductor import QuadraticPdeoptStationaryCoerciveReductor
from pdeopt.TR import TR_algorithm
from pdeopt.relaxed_TR import Relaxed_TR_algorithm
from pymor.parameters.functionals import MinThetaParameterFunctional

set_defaults({'pymor.operators.constructions.induced_norm.raise_negative': False})
set_defaults({'pymor.operators.constructions.induced_norm.tol': 1e-20})

ce = MinThetaParameterFunctional(opt_fom.primal_model.operator.coefficients, mu_bar)

# ## NCD corrected BFGS Method (KMSOV'20)
counter.reset_counters()

tic = time.time()
params = [mu]

if ('Method_R_TR' in optimization_methods and 'Method_RB' in optimization_methods) or \
    'All' in optimization_methods:
    print("\n_________________Relaxed TR NCD BFGS_____________________\n")
    # make sure to use the correct config
    opt_fom = opt_fom.with_(use_corrected_functional=True)
    opt_fom = opt_fom.with_(adjoint_approach=True)

    RBbasis, dual_RBbasis = build_initial_basis(
        opt_fom, params, build_sensitivities=False)
    
    pdeopt_reductor = \
        QuadraticPdeoptStationaryCoerciveReductor(opt_fom, 
                                                  RBbasis, dual_RBbasis, 
                                                  opt_product=opt_fom.opt_product,
                                                  coercivity_estimator=ce,
                                                  reductor_type=relaxed_reductor_type,
                                                  mu_bar=mu_bar)

    opt_rom = pdeopt_reductor.reduce()

    tictoc = time.time() - tic

    TR_parameters = {'Qian-Grepl_subproblem': Qian_Grepl_subproblem, 'beta': beta,
                 'safety_tolerance': safety_tol,
                 'radius': 0.1, 'FOC_tolerance': FOC_tolerance, 
                 'sub_tolerance': sub_tolerance,
                 'max_iterations': max_it, 'max_iterations_subproblem': max_it_sub, 
                 'max_iterations_armijo': max_it_arm,
                 'initial_step_armijo': init_step_armijo, 
                 'armijo_alpha': armijo_alpha, 
                 'epsilon_i': epsilon_i, 
                 'control_mu': control_mu,
                 'starting_parameter': mu, 
                 'opt_method': 'BFGSMethod'}

    extension_params = {'Enlarge_radius': True, 'timings': True, 
                        'opt_fom': opt_fom, 'return_data_dict': True}

    mus_ntr8, times_ntr8, Js_ntr8, FOC_ntr8, data_ntr8 = \
        Relaxed_TR_algorithm(opt_rom, pdeopt_reductor, parameter_space,
                             TR_parameters, extension_params, skip_estimator=skip_estimator)
    
    times_full_ntr8_actual, J_error_ntr8_actual, mu_error_ntr8_actual, FOC_ntr8_actual = \
        compute_errors(opt_fom, parameter_space, J_start, J_opt, mu, mu_opt_as_array,
                       mus_ntr8, Js_ntr8, times_ntr8, tictoc, FOC_ntr8, pool=pool)

In [12]:
if ('Method_R_TR' in optimization_methods and 'Method_RB' in optimization_methods) \
    or 'All' in optimization_methods:
    print("\n_________________Relaxed TR NCD BFGS_____________________\n")
    R_TRNCDRB_dict = counter.print_result(True)
    # print_RB_result(R_TRNCDRB_dict)
    print_iterations_and_walltime(len(times_full_ntr8_actual), times_full_ntr8_actual[-1])
    print('mu_error: ', mu_error_ntr8_actual[-1])
    subproblem_time = data_ntr8['total_subproblem_time']
    print(f'further timings:\n subproblem:  {subproblem_time:.3f}')

counter.reset_counters()

In [13]:
tic = time.time()
params = [mu]

# TODO: FIX THE VERSION OF PYMOR HERE !!! 
if 0 and 'Method_RB' in optimization_methods or 'All' in optimization_methods:
    print("\n_________________TR NCD BFGS_____________________\n")
    # make sure to use the correct config
    opt_fom = opt_fom.with_(use_corrected_functional=True)
    opt_fom = opt_fom.with_(adjoint_approach=True)

    RBbasis, dual_RBbasis = build_initial_basis(
        opt_fom, params, build_sensitivities=False)
    
    pdeopt_reductor = QuadraticPdeoptStationaryCoerciveReductor(opt_fom, 
                                                RBbasis, dual_RBbasis, 
                                                opt_product=opt_fom.opt_product,
                                                coercivity_estimator=ce,
                                                reductor_type=reductor_type, mu_bar=mu_bar)

    opt_rom = pdeopt_reductor.reduce()
    
    tictoc = time.time() - tic

    TR_parameters = {'Qian-Grepl_subproblem': Qian_Grepl_subproblem, 'beta': beta,
                 'safety_tolerance': safety_tol,
                 'radius': radius, 'FOC_tolerance': FOC_tolerance, 
                 'sub_tolerance': sub_tolerance,
                 'max_iterations': max_it, 'max_iterations_subproblem': max_it_sub, 
                 'max_iterations_armijo': max_it_arm,
                 'initial_step_armijo': init_step_armijo, 
                 'armijo_alpha': armijo_alpha, 
                 'epsilon_i': epsilon_i, 
                 'control_mu': control_mu,
                 'starting_parameter': mu, 
                 'opt_method': 'BFGSMethod'}

    extension_params = {'Enlarge_radius': True, 'timings': True, 
                        'opt_fom': opt_fom, 'return_data_dict': True}

    mus_8, times_8, Js_8, FOC_8, data_8 = TR_algorithm(opt_rom, pdeopt_reductor,
                                                       parameter_space, TR_parameters,
                                                       extension_params)
    
    times_full_8_actual, J_error_8_actual, mu_error_8_actual, FOC_8_actual = \
        compute_errors(opt_fom, parameter_space, J_start, J_opt, mu, mu_opt_as_array,
                       mus_8, Js_8, times_8, tictoc, FOC_8, pool=pool)


if 0 and 'Method_RB' in optimization_methods or 'All' in optimization_methods:
    plt.semilogy(times_full_8_actual, FOC_8_actual)

In [14]:
if 0 and 'Method_RB' in optimization_methods or 'All' in optimization_methods:
    print("\n_________________TR NCD BFGS_____________________\n")
    TRNCDRB_dict = counter.print_result(True)
    # print_RB_result(TRNCDRB_dict)
    print_iterations_and_walltime(len(times_full_8_actual), times_full_8_actual[-1])
    print('mu_error: ', mu_error_8_actual[-1])
    subproblem_time = data_8['total_subproblem_time']
    print(f'further timings:\n subproblem:  {subproblem_time:.3f}')

counter.reset_counters()

# LRBMS

In [15]:
from pdeopt_discretizer import discretize_quadratic_pdeopt_with_iplrb

In [16]:
N = 5.
macro_diameter = np.sqrt(2)/N
# automatically detect how many refinements are needed
refinements = int(np.log2(n/N))

ipl_opt_fom, data, mu_bar = \
    discretize_quadratic_pdeopt_with_iplrb(global_problem,
                                           macro_diameter,
                                           refinements=refinements,
                                           symmetry_factor=1.,
                                           weight_parameter=None,
                                           penalty_parameter=16.,
                                           weights=weights.copy(),
                                           domain_of_interest=domain_of_interest,
                                           desired_temperature=desired_temperature,
                                           mu_for_u_d=mu_for_u_d,
                                           mu_for_tikhonov=mu_for_tikhonov,
                                           pool=pool,
                                           counter=None,
                                           store_in_tmp=store_in_tmp,
                                           coarse_J=coarse_J,
                                           use_fine_mesh=use_fine_mesh,
                                           aFine_constructor=local_problem_constructer,
                                           u_d=u_d,
                                           print_on_ranks=print_on_ranks)

Discretizing ipld3g model ...done with 25 subdomains and 5 refinements
constant  0.00220304218636278


In [17]:
macro_grid = data['macro_grid']
dd_grid = data['dd_grid']
local_spaces = data['local_spaces']

In [18]:
J_opt_ipl = ipl_opt_fom.output_functional_hat(mu_opt)
J_start_ipl = ipl_opt_fom.output_functional_hat(mu)
print('Optimal J IPL: ', J_opt_ipl)
print('Optimal J FEM: ', J_opt)
print('Starting J IPL: ', J_start_ipl)
print('Starting J FEM: ', J_start)

Optimal J IPL:  1.0
Optimal J FEM:  1.0
Starting J IPL:  1.0507371060380521
Starting J FEM:  1.0508730423458459


In [19]:
TR_parameters = {'radius': 1.e18, 'sub_tolerance': FOC_tolerance, 
                 'max_iterations_subproblem': 500,
                 'starting_parameter': mu,
                 'epsilon_i': epsilon_i,
                 'max_iterations_armijo': max_it_arm,
                 'initial_step_armijo': init_step_armijo,
                 'armijo_alpha': armijo_alpha,
                 'full_order_model': True}
counter.reset_counters()

if 'BFGS_IPL' in optimization_methods or 'All' in optimization_methods:
    print("\n________________IPL FOM BFGS________________________\n")
    muoptfom,_,_,_, times_ipl_FOM, mus_ipl_FOM, Js_ipl_FOM, FOC_ipl_FOM = \
            solve_optimization_subproblem_BFGS(ipl_opt_fom, parameter_space, mu,
                                               TR_parameters, timing=True, FOM=True)
    times_full_ipl_FOM, J_error_ipl_FOM, mu_error_ipl_FOM, FOC = \
            compute_errors(ipl_opt_fom, parameter_space, J_start, J_opt, mu, mu_opt_as_array, 
                           mus_ipl_FOM, Js_ipl_FOM, times_ipl_FOM, 0, FOC_ipl_FOM)
    times_full_ipl_FOM = times_full_ipl_FOM[1:]


________________IPL FOM BFGS________________________

Starting projected BFGS method
Starting parameter {diffusion: [2.251066014107722, 3.1609734803264744, 1.0003431244520347, 1.9069977178955193, 1.440267672451339, 1.2770157843063934, 1.5587806341330128, 2.036682181129143, 2.19030242269201, 2.616450202010071, 2.2575835432098845, 3.0556585011902784, 1.6133567491945522, 3.6343523091728365, 1.0821627795937785, 3.011402530535207, 2.251914407101381, 2.676069485337255, 1.4211608157857012, 1.5943044672546365, 3.40223370602661, 3.9047847271581926, 1.9402725344777285, 3.0769678470079422], low_diffusion: [1.1752778304592075, 1.1789213327007695, 1.0170088422739556, 1.0078109566465765, 1.0339660839129137, 1.1756285006858826, 1.01966936676661, 1.0842215250010103]}
Step [2.25266067 3.16293455 1.00689317 1.91330063 1.44735949 1.28332168
 1.56309354 2.04053477 2.19438722 2.62015932 2.25875791 3.05907024
 1.61708357 3.63430683 1.08692011 3.01413372 2.25385599 2.67524158
 1.42310213 1.59604082 3.403291

Step [2.63730931 3.88926514 3.51320804 4.         4.         3.35596106
 3.95503799 4.         3.97890994 4.         1.26721485 4.
 3.44923104 2.0538026  3.74343166 3.61798837 3.3269572  1.24985495
 1.67222489 1.67151073 3.28173032 2.79767996 1.24604712 1.05331689
 1.1231543  1.12040384 1.2        1.2        1.2        1.12385375
 1.2        1.2       ], functional 1.000416173519301 , FOC condition 0.0010765735500409105
Step [2.6437366  3.89822525 3.54332457 3.99998962 3.99999015 3.38351592
 3.97801557 3.99999981 3.99882775 4.         1.26681961 4.
 3.46853358 2.04746662 3.76928008 3.64063068 3.33765727 1.24175785
 1.67991294 1.67784867 3.28510792 2.79363973 1.24321067 1.04053993
 1.13715983 1.13403876 1.19991252 1.19989768 1.2        1.13796424
 1.2        1.2       ], functional 1.0004013137499248 , FOC condition 0.0010391441807089108
Step [2.64989417 3.90754266 3.57469134 4.         4.         3.41132857
 4.         3.97601518 4.         4.         1.26312612 4.
 3.49045949 2.037582

Step [2.56800492 3.86497533 3.4038637  3.99735222 3.99999851 3.16185021
 3.79384886 3.97750266 3.99069106 4.         1.02451957 4.
 3.68821993 1.85419019 4.         3.91280024 3.55134688 1.15245074
 1.88565935 1.86121133 3.44734453 2.83538091 1.29456582 1.04639519
 1.08482331 1.10757828 1.10090194 1.10375676 1.19257685 1.18841198
 1.2        1.14213003], functional 1.0000516889183508 , FOC condition 0.00032895024156388787
Step [2.56835625 3.8667118  3.40888064 3.99607029 4.         3.16477208
 3.8076185  4.         3.97838354 3.99999982 1.02424894 4.
 3.69768806 1.84992993 4.         3.92780119 3.55841813 1.14944797
 1.89233838 1.8671474  3.45204946 2.83609077 1.29544292 1.0378988
 1.08596933 1.10132942 1.09351341 1.09968278 1.19647606 1.1926379
 1.2        1.13879372], functional 1.000048951187604 , FOC condition 0.000318416238244637
Step [2.56680001 3.86617484 3.40627327 3.99972476 3.99999532 3.16025934
 3.82185558 4.         3.95821894 4.         1.02288238 4.
 3.70297325 1.84528349

In [20]:
if 'BFGS_IPL' in optimization_methods or 'All' in optimization_methods:
    print("\n________________IPL FOM BFGS________________________\n")
    IPL_BFGS_dict = counter.print_result(True)
    # print_RB_result(BFGS_dict)
    print_iterations_and_walltime(len(times_full_ipl_FOM), times_full_ipl_FOM[-1])
    print('mu_error: ', mu_error_ipl_FOM[-1]) 
counter.reset_counters()


________________IPL FOM BFGS________________________


FEM solves:   0
RB solves:    0


outer iterations: 58
total walltime:   1258.47 seconds
mu_error:  0.09373771348267972


In [23]:
from pdeopt_reductor import QuadraticPdeoptStationaryCoerciveLRBMSReductor
tic = time.time()

if ('Method_R_TR' in optimization_methods and 'Method_LRBMS' in optimization_methods) \
        or 'All' in optimization_methods:
    print("\n_________________Relaxed TR LRBMS BFGS_____________________\n")
    
    # make sure to use the correct config
    iplrb_opt_fom = ipl_opt_fom.with_(use_corrected_functional=False,
                                      adjoint_approach=False)
    
    print('constructing reductor ...')
    pdeopt_reductor = QuadraticPdeoptStationaryCoerciveLRBMSReductor(
        ipl_opt_fom, f, dd_grid=dd_grid, opt_product=iplrb_opt_fom.opt_product,
        coercivity_estimator=ce, reductor_type='non_assembled',
        mu_bar=mu_bar, parameter_space=parameter_space,
        pool=pool, optional_enrichment=False, store_in_tmp=store_in_tmp,
        use_fine_mesh=use_fine_mesh,
        print_on_ranks=print_on_ranks
    )
    print('enriching ...')
    
    pdeopt_reductor.add_global_solutions(mu)

    print('construct ROM ...')
    opt_rom = pdeopt_reductor.reduce()

    tictoc = time.time() - tic

    TR_parameters = {'Qian-Grepl_subproblem': Qian_Grepl_subproblem, 'beta': beta,
                 'safety_tolerance': safety_tol,
                 'radius': 0.1, 'FOC_tolerance': FOC_tolerance, 
                 'sub_tolerance': sub_tolerance,
                 'max_iterations': max_it, 'max_iterations_subproblem': max_it_sub, 
                 'max_iterations_armijo': max_it_arm,
                 'initial_step_armijo': init_step_armijo, 
                 'armijo_alpha': armijo_alpha, 
                 'epsilon_i': epsilon_i, 
                 'control_mu': control_mu,
                 'starting_parameter': mu, 
                 'opt_method': 'BFGSMethod'}

    extension_params = {'Enlarge_radius': True, 'timings': True, 
                        'opt_fom': opt_fom, 'return_data_dict': True}

    mus_rlrb, times_rlrb, Js_rlrb, FOC_rlrb, data_rlrb = \
        Relaxed_TR_algorithm(opt_rom, pdeopt_reductor, parameter_space, TR_parameters,
                             extension_params, skip_estimator=skip_estimator)
    
    times_full_rlrb_actual, J_error_rlrb_actual, mu_error_rlrb_actual, FOC_rlrb_actual = \
        compute_errors(ipl_opt_fom, parameter_space, J_start, J_opt, mu, mu_opt_as_array,
                       mus_rlrb, Js_rlrb, times_rlrb, tictoc, FOC_rlrb, pool=pool)


_________________Relaxed TR LRBMS BFGS_____________________

constructing reductor ...
initializing roms ...

Initialization took 0.0141s
enriching ...
construct ROM ...
constructing ROM ... ... Model has been constructed ... length of bases are [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 ... construction took 0.3374s
starting parameter {diffusion: [2.251066014107722, 3.1609734803264744, 1.0003431244520347, 1.9069977178955193, 1.440267672451339, 1.2770157843063934, 1.5587806341330128, 2.036682181129143, 2.19030242269201, 2.616450202010071, 2.2575835432098845, 3.0556585011902784, 1.6133567491945522, 3.6343523091728365, 1.0821627795937785, 3.011402530535207, 2.251914407101381, 2.676069485337255, 1.4211608157857012, 1.5943044672546365, 3.40223370602661, 3.9047847271581926, 1.9402725344777285, 3.0769678470079422], low_diffusion: [1.1752778304592075, 1.1789213327007695, 1.0170088422739556, 1.0078109566465765, 1.0339660839129137, 1.1756285006858826, 1.019669

In [24]:
if ('Method_R_TR' in optimization_methods and 'Method_LRBMS' in optimization_methods) \
    or 'All' in optimization_methods:
    print("\n_________________TR LRBMS BFGS_____________________\n")
    TRLRBMS_dict = counter.print_result(True)
    # print_RB_result(TRLRBMS_dict)
    print_iterations_and_walltime(len(times_full_rlrb_actual), times_full_rlrb_actual[-1])
    print('mu_error: ', mu_error_rlrb_actual[-1])
    subproblem_time = data_rlrb['total_subproblem_time']
    print(f'further timings:\n subproblem:  {subproblem_time:.3f}')

counter.reset_counters()


_________________TR LRBMS BFGS_____________________


FEM solves:   0
RB solves:    0


outer iterations: 4
total walltime:   880.90 seconds
mu_error:  0.012873979335380464
further timings:
 subproblem:  664.442
